# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [3]:
import pandas as pd

reviews_df = pd.read_csv("reviews_sample.csv")
recipes_df = pd.read_csv("recipes_sample.csv")

# нужные столбцы
recipes_df = recipes_df[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]

print(recipes_df.head())

      id                                      name  minutes   submitted  \
0  44123     george s at the cove  black bean soup       90  2002-10-25   
1  67664        healthy for them  yogurt popsicles       10  2003-07-26   
2  38798              i can t believe it s spinach       30  2002-08-29   
3  35173                      italian  gut busters       45  2002-07-27   
4  84797  love is in the air  beef fondue   sauces       25  2004-02-23   

                                         description  n_ingredients  
0  an original recipe created by chef scott meska...           18.0  
1  my children and their friends ask for my homem...            NaN  
2            these were so go, it surprised even me.            8.0  
3  my sister-in-law made these for us at a family...            NaN  
4  i think a fondue is a very romantic casual din...            NaN  


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [26]:
import pandas as pd
from openpyxl import Workbook

reviews_df = pd.read_csv("reviews_sample.csv")
recipes_df = pd.read_csv("recipes_sample.csv")

# случайные 5% строк из каждой таблицы
recipes_sample = recipes_df.sample(frac=0.05, random_state=42)

# новый файл Excel
with pd.ExcelWriter("recipes.xlsx", engine='openpyxl') as writer:
    # запись с рецептами на лист "Рецепты"
    recipes_sample.to_excel(writer, sheet_name="Рецепты", index=False)
    
    # запись  с отзывами на новый лист "Отзывы"
    reviews_sample.to_excel(writer, sheet_name="Отзывы", index=False)

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [5]:
import pandas as pd
import xlwings as xw

wb = xw.Book("recipes.xlsx")
sheet = wb.sheets["Рецепты"]
recipes_df = sheet.range("A1").expand().options(pd.DataFrame).value

# вычисление времени выполнения в секундах
recipes_df['seconds_assign'] = recipes_df['minutes'] * 60

# + нового столбца seconds_assign в файл Excel
sheet.range('I1').options(index=False).value = recipes_df['seconds_assign']

wb.save()
wb.close()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [10]:
import xlwings as xw

wb = xw.Book("recipes.xlsx")
sheet = wb.sheets["Рецепты"]

# + формула для столбца seconds_formula
sheet.range('J1').value = 'seconds_formula'
# столбец минуты умножаем на 60
sheet.range('J2').formula = "=C2*60"
# заполнение ячеек формулой сверху
sheet.range('J2').api.AutoFill(sheet.range('J2:J{}'.format(sheet.range('C1').end('down').row)).api, 0)

wb.save()
wb.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [11]:
import xlwings as xw

wb = xw.Book("recipes.xlsx")
sheet = wb.sheets["Рецепты"]

# + формула для столбца seconds_formula
sheet.range('K1').value = 'seconds_formula'
sheet.range('K2').formula = "=C2*60"
sheet.range('K2').api.AutoFill(sheet.range('K2:K{}'.format(sheet.range('C1').end('down').row)).api, 0)

# форматирует заголовок столбца жирным и выравнивает по центру
header_range = sheet.range('K1')
header_range.api.Font.Bold = True
header_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

wb.save()
wb.close()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [14]:
import xlwings as xw

wb = xw.Book("recipes.xlsx")
sheet = wb.sheets["Рецепты"]

# + формула для столбца seconds_formula
sheet.range('K1').value = 'seconds_formula'
sheet.range('K2').formula = "=C2*60"
sheet.range('K2').api.AutoFill(sheet.range('K2:K{}'.format(sheet.range('C1').end('down').row)).api, 0)

# раскраска ячеек
minutes_range = sheet.range('C2:C{}'.format(sheet.range('C1').end('down').row))
for cell in minutes_range:
    if cell.value < 5:
        cell.color = (0, 255, 0)  # зленый
    elif 5 <= cell.value <= 10:
        cell.color = (255, 255, 0)  # желтый
    else:
        cell.color = (255, 0, 0)  # красный

# форматирование заголовков 
header_range = sheet.range('K1')
header_range.api.Font.Bold = True
header_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

wb.save()
wb.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [12]:
import xlwings as xw

def n_reviews_column():
    try:
        wb = xw.Book("recipes.xlsx")
        sheet_recipes = wb.sheets["Рецепты"]
        
        # последняя строка на листе "Рецепты"
        last_row = sheet_recipes.range('A' + str(sheet_recipes.cells.last_cell.row)).end('up').row
        
        # заголовок
        sheet_recipes.range('G1').value = 'n_reviews'
        
        # формула Excel для подсчета кол-ва отзывов
        for i in range(2, last_row + 1):
            recipe_id = sheet_recipes.range('A' + str(i)).value
            reviews_count_formula = f'=COUNTIF(Отзывы!A:A, {recipe_id})'
            sheet_recipes.range('G' + str(i)).formula = reviews_count_formula
        
        wb.save()
        wb.close()

# вызов 
add_n_reviews_column()


SyntaxError: expected 'except' or 'finally' block (1735026232.py, line 24)

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
import xlwings as xw

def validate():
    wb = xw.Book("recipes.xlsx")
    sheet_reviews = wb.sheets["Отзывы"]
    sheet_recipes = wb.sheets["Рецепты"]
    
    # определение последней строки в отзывах 
    last_row = sheet_reviews.range('A' + str(sheet_reviews.cells.last_cell.row)).end('up').row
    
    # проверка каждой строки
    for i in range(2, last_row + 1):  # пропуск заголовка
        rating = sheet_reviews.range('B' + str(i)).value
        recipe_id = sheet_reviews.range('A' + str(i)).value
        
        # проверка рейтинга
        if rating is None or not (0 <= rating <= 5):
            sheet_reviews.range('B' + str(i)).color = (255, 0, 0)  # красный цвет для рейтинга
        
        # Проверка наличия рецепта на листе "Рецепты"
        if recipe_id not in sheet_recipes.range('A:A').value:
            sheet_reviews.range('A' + str(i)).color = (255, 0, 0)  # красный цвет для ID
    
    wb.save()
    wb.close()

# вызов
validate()


9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 